In [78]:
#Generates a .csv file from data from https://simplemaps.com/data/gb-cities
#Note - this code is specific to this website
import csv
file = open("gb.csv")
reader = csv.reader(file)
lines = len(list(reader))
file.close()
file = open("List of cities.csv", "w")
file.close()
for row in range(1,lines-1):
    file = open("gb.csv")
    reader = csv.reader(file)
    cityName = list(reader)[row][0]
    file.close()
    file = open("List of cities.csv", "a")
    file.write(cityName + ",")
    file.close()

file = open("gb.csv")
reader = csv.reader(file)
cityName = list(reader)[lines-1][0]
file.close()
file = open("List of cities.csv", "a")
file.write(cityName)
file.close()

In [25]:
#main
import csv
file = open("List of cities.csv", "r")
reader = csv.reader(file)
l = list(reader) #creates an array of cities
file.close()

file = open("failedQueries.txt", "w") #deletes file if already exists
file.close()

deleteDB() #deletes contents of db table if already exists

import requests

for i in range(0,100): #len(l[0])
    geoJ = getGeoJ(i)
    tempGeoJ = geoJ #stores the geoJ string in a temporary variable which is changed throughout the loop
    fail = True
    while True:
        start = tempGeoJ.find("{'type': 'Polygon', 'coordinates': [[") #This substring depends on the format of the geoJ file! e.g. "" may be used instead of ''
        if start == -1:
            start = tempGeoJ.find("{'type': 'MultiPolygon', 'coordinates': [[")
        end = tempGeoJ.find("]]}") + 3
        if start == -1: 
            if fail == True: 
                failLog()
            break
                
        displayName = getDisplayName(tempGeoJ)
        region = getRegion(displayName)
        cityName = getCityName(displayName)
        parsedGeoJ = tempGeoJ[start:end] #creates parsed geoJ file
        file = open("C:/Users/Toan/Documents/CityGeoJSON/United Kingdom/GeoJSON files/" + displayName + ".json", "w") 
        file.write(parsedGeoJ)
        file.close()
        insertDB(cityName, region, displayName)
        tempGeoJ = tempGeoJ[end:]
        fail = False #If the loop has executed at least once without a failure, it does not need to be logged as a failure.

In [11]:
def getDisplayName(tempGeoJ): #This finds the name of the city, which the file is named after
    displayName = ""
    nameStart = tempGeoJ.find("display_name") + 16
    char = tempGeoJ[nameStart] 
    while char != "'":
        if char != "/":                
            displayName = displayName + char
        nameStart += 1
        char = tempGeoJ[nameStart] 
    return displayName

In [12]:
def getRegion(displayName): #fetches the part of the display name between the first two commas
    region = ""
    for i in range(len(displayName)):
        if displayName[i] == ",":
            x = 1
            while displayName[i+x] != ",":
                region = region + displayName[i+x]
                x = x+1
            break
    return region

In [13]:
def getCityName(displayName): #fetches the part of the display name up until the first comma
    cityName = ""
    i = 0
    while displayName[i] != ",":
        cityName = cityName + displayName[i]
        i = i+1
    return cityName

In [14]:
def getGeoJ(i):
    searchCity = l[0][i]  #As it is from a .csv file, the array is 2D, i.e. [[London,Birmingham...,Broughton]] - hence 2 indexes are required
    api_url = "https://nominatim.openstreetmap.org/search.php?city=" + searchCity + "&country=United+Kingdom&polygon_geojson=1&format=json"
    response = requests.get(api_url) #retrieves geoJSON file from Nominatim API
    return str(response.json())


In [15]:
def failLog():
    file = open("failedQueries.txt", "a")
    file.write(l[0][i])
    file.write(geoJ)
    file.write("\n")
    file.write("Error: Request did not return Polygon")
    file.write("\n")
    file.close()

In [16]:
def insertDB(cityName, region, fileName):
    import pyodbc

    filePath = "'C:\\Users\\Toan\\Documents\\CityGeoJSON\\United Kingdom\\GeoJSON files\\" + fileName + ".json'"
    
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=HUNG-LAPTOP\TOAN;'
                          'Database=CityGeoJSON;'
                          'Trusted_Connection=yes;')

    cursor = conn.cursor()

    try:
        cursor.execute('''
                    INSERT INTO CityGeoJSON.dbo.CityGeoJSON (CountryName, CountryIsoCode, Region, CityName, GeoJson)
                    SELECT 'United Kingdom', 'GBR', ?, ?, * FROM OPENROWSET(BULK ''' + filePath + ''',SINGLE_CLOB) AS DATA;
                    ''',
                   (region, cityName)
                  )
    except pyodbc.IntegrityError:
        file = open("failedQueries.txt", "a")
        file.write(cityName + region)
        file.write(parsedGeoJ)
        file.write("\n")
        file.write("Error: Duplicate city+region")
        file.write("\n")
        file.close()
        
    conn.commit()

In [17]:
def deleteDB():
    import pyodbc
    
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=HUNG-LAPTOP\TOAN;'
                          'Database=CityGeoJSON;'
                          'Trusted_Connection=yes;')
    

    cursor = conn.cursor()
    
    cursor.execute('''
                    DELETE CityGeoJSON.dbo.CityGeoJSON
                    ''')
    conn.commit()